# 🧠 Merchant Sales Analysis with AI
This notebook includes sales insights such as best-selling items, sales trends, and an integration with Gemini AI to provide business recommendations.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import google.generativeai as genai
from datetime import datetime


In [ ]:
# Load CSV files
def load_csv_files():
    items = pd.read_csv('items.csv')
    keywords = pd.read_csv('keywords.csv')
    merchants = pd.read_csv('merchant.csv')
    transactions = pd.read_csv('transaction_data.csv')
    transaction_items = pd.read_csv('transaction_items.csv')
    return items, keywords, merchants, transactions, transaction_items

In [ ]:
# Merge datasets
def merge_data(items, keywords, merchants, transactions, transaction_items):
    merged = transactions.merge(merchants, on='merchant_id', how='left')
    merged = merged.merge(transaction_items, on='order_id', how='left')
    merged = merged.merge(items, on='item_id', how='left')
    merged['order_time'] = pd.to_datetime(merged['order_time'], errors='coerce')
    return merged

In [ ]:
# Data Analysis Functions
def top_items(df):
    print("Top 5 Best-Selling Items:\n", df['item_name'].value_counts().head(5))

def sales_trends(df):
    df['day'] = df['order_time'].dt.date
    trend = df.groupby('day')['order_value'].sum()
    trend.plot(title="Sales Trend by Day", figsize=(10, 5))
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def peak_sales_hours(df):
    df['hour'] = df['order_time'].dt.hour
    print("\nPeak Sales Hours:\n", df['hour'].value_counts().sort_index())

def best_by_category(df):
    print("\nBest-Selling Item per Category:")
    print(df.groupby('category')['item_name'].agg(lambda x: x.value_counts().index[0]))

def compare_months(df):
    df['month'] = df['order_time'].dt.to_period("M")
    print("\nMonthly Sales Comparison:\n", df.groupby('month')['order_value'].sum())

In [ ]:
# Gemini AI Summary
def gemini_ai_summary(summary_text):
    genai.configure(api_key="your_api_key_here")  # Replace with your API key
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(f"Sales Summary: {summary_text}\nWhat business advice would you give this merchant?")
    print("\nGemini Response:\n", response.text)

In [ ]:
# Run all
items, keywords, merchants, transactions, transaction_items = load_csv_files()
df = merge_data(items, keywords, merchants, transactions, transaction_items)

top_items(df)
sales_trends(df)
peak_sales_hours(df)
best_by_category(df)
compare_months(df)

summary = f"Top items: {', '.join(df['item_name'].value_counts().head(3).index)}. Total sales: {df['order_value'].sum()}"
gemini_ai_summary(summary)